In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data):
    self.size = 1 << (len(data) - 1).bit_length()
    self.L = [0] * (2 * self.size)
    self.lazy = [0] * (2 * self.size)
    self.L[self.size:self.size + len(data)] = data
    for i in reversed(range(self.size)):
      self.L[i] = self.L[i*2] ^ self.L[i*2+1]

  def push(self, i, s, e) :
    if not self.lazy[i] : return

    # XOR은 짝수번 수행하면 그 결과가 같으므로 실행하지 않아도 된다.
    # self.L[i] += (e - s) * self.lazy[i]
    if i < self.size :
      self.lazy[i*2] ^= self.lazy[i]
      self.lazy[i*2+1] ^= self.lazy[i]
    else :
      self.L[i] ^= self.lazy[i]
    self.lazy[i] = 0

  def query(self, l, r):
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.push(i, nl, nr)
    
    if r <= nl or nr <= l : return 0
    if l <= nl and nr <= r : return self.L[i]
    mid = (nl + nr) // 2
    return self._query(l, r, i*2, nl, mid) ^ self._query(l, r, i*2+1, mid, nr)
  
  def update(self, l, r, x):
    self._update(l, r, x, 1, 0, self.size)

  def _update(self, l, r, x, i, nl, nr) :
    self.push(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] += x
      self.push(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._update(l, r, x, i*2, nl, mid)
    self._update(l, r, x, i*2+1, mid, nr)
    self.L[i] = self.L[i*2] ^ self.L[i*2+1]

def sol() :
  N = int(input())
  L = [*map(int, input().split())]
  lst = LazySegmentTree(L)

  M = int(input())
  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 1 :
      s, e, x = l
      s, e = sorted([s, e])
      lst.update(s, e+1, x)
    elif q == 2 :
      s, e = sorted(l)
      ans.append(lst.query(s, e+1))
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

### XOR($\oplus$) 연산의 다양한 성질
- $a \oplus a = 0$ 이다. 
- $a \oplus 0 = a$ 이다.
- 그리고 또 다른 값 $b$ 가 있다고 하자, 이때 $a$ 에 대해 $\oplus b$ 연산을 짝수번 수행하면 원본 값인 $a$ 가 나온다.
  - $a \oplus b \oplus b = a \oplus (b \oplus b) = a \oplus 0 = a$

### 풀이
- 1번 쿼리를 $Q_1(a, b, w) : L_a := L_a \oplus w, L_{a+1} := L_{a+1} \oplus w, ..., L_b := L_b \oplus w$ 라고 하자.
- 3번 성질에 의해서 리프노드가 아닌 노드에 대해서는 $\oplus w$ 연산에 대한 정보가 없어진다. 
  - 따라서 리프노드에 대해서만 $\oplus w$ 연산을 수행하면 된다.
  - 대신 조상 노드들에 대해서 리프노드의 연산 결과를 저장해야 한다.

- 좀더 깔끔한 템플릿을 사용한 구현.

In [ ]:
import io, os, sys
from operator import xor
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data, f=lambda x, y: x+y, fl=lambda i, x, cnt: i+x*cnt, fll=lambda x, y: x+y, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(data)
    self.L = [default] * self.len + data
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.L[i] = self.f(self.L[i*2], self.L[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _push(self, idx, val) :
    self.L[idx] = self.fl(self.L[idx], val, self.cnt[idx])
    if idx < self.len :
      self.lazy[idx] = self.fll(self.lazy[idx], val)

  def _build(self, idx) :
    while idx :
      if idx < self.len :
        self._apply(idx)
        self.L[idx] = self.f(self.L[idx*2], self.L[idx*2+1])
      idx >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.L[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.L[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)

def sol() :
  N = int(input())
  L = [*map(int, input().split())]
  lst = LazySegmentTree(L, f=xor, fl=lambda i, x, cnt: i^x if cnt & 1 else i, fll=xor)

  M = int(input())
  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 1 :
      s, e, x = l
      lst.update(s, e, x)
    elif q == 2 :
      s, e = l
      ans.append(lst.query(s, e))
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()